<a href="https://colab.research.google.com/github/masadream/data_analytics/blob/master/Causal_Inference_studysession/Ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.4.1

##### モジュール準備


In [0]:
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import ttest_ind

##### データセットダウンロード

In [0]:
email_data = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

##### 女性向けメール配信データをデータセットから除外

In [0]:
male_df = email_data[email_data["segment"] != "Womens E-Mail"]

##### 介入を表すtreatment変数を追加

In [4]:
male_df["treatment"] = np.where(male_df["segment"] == "Mens E-Mail", 1, 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# 1.4.2

##### treatmentごとの簡単な集計結果

In [5]:
male_df.groupby("treatment").agg({"conversion" : "mean", "spend" : "mean", "treatment" : "count"})

,conversion,spend,treatment
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307


##### 元ソースではRでStudentのt検定・両側検定をやっているので、それにしたがう
https://cran.r-project.org/doc/manuals/r-release/fullrefman.pdf
https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.ttest_ind.html#statsmodels.stats.weightstats.ttest_ind


In [6]:
mens_mail = male_df[male_df["treatment"] == 1]
mens_mail = mens_mail["spend"]
no_mail = male_df[male_df["treatment"] == 0]
no_mail = no_mail["spend"]

rct_ttest = ttest_ind(mens_mail, no_mail, alternative = 'two-sided', usevar = 'pooled')
rct_ttest

(5.300090294465455, 1.163200872605976e-07, 42611.0)

# 1.4.3

##### randomseedの固定

In [0]:
np.random.seed(1)

##### 初期値

In [8]:
biased_data = male_df
biased_data["obs_rate_c"] = 0.5
biased_data["obs_rate_t"] = 0.5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##### バイアスデータの作成

In [9]:
biased_data["obs_rate_c"] = np.where((biased_data["history"] > 300) | (biased_data["recency"] < 6) | (biased_data["channel"] == "Multichannel"), biased_data["obs_rate_c"], 1)

biased_data["obs_rate_t"] = np.where((biased_data["history"] > 300) | (biased_data["recency"] < 6) | (biased_data["channel"] == "Multichannel"), 1, biased_data["obs_rate_t"])

biased_data["random_number"] = np.random.rand(len(biased_data))

biased_data = biased_data[(biased_data["treatment"] == 0 ) & (biased_data["random_number"] < biased_data["obs_rate_c"]) | (biased_data["treatment"] == 1) & (biased_data["random_number"] < biased_data["obs_rate_t"])]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

##### バイアスデータのtreatmentごとの簡単な集計

In [10]:
biased_data.groupby("treatment").agg({"conversion" : "mean", "spend" : "mean", "treatment" : "count"})

,conversion,spend,treatment
treatment,,,
0,0.005673,0.639339,14806
1,0.013079,1.468905,17203


##### バイアスデータでのt検定実施

In [12]:
mens_mail_biased = biased_data[biased_data["treatment"] == 1]
mens_mail_biased = mens_mail_biased["spend"]
no_mail_biased = biased_data[biased_data["treatment"] == 0]
no_mail_biased = no_mail_biased["spend"]

rct_ttest_biased = ttest_ind(mens_mail_biased, no_mail_biased, alternative = 'two-sided', usevar = 'pooled')
rct_ttest_biased

(4.857590542261104, 1.1938371418318866e-06, 32007.0)